# Pre train and Fine-tune with Affect datasets

This example shows a very basic usage case of MultiBench. 
In particular, it demonstrates how to use MultiBench with the affective computing datasets, and how to use it with a very simple fusion model.

## Previous steps

To begin, let's clone the repo and setup our interpreter to run commands inside the folder.

In [ ]:
!git clone https://github.com/pliang279/MultiBench.git
%cd MultiBench

Try to download the data file for MOSI using the below command. If this does not work for you, please download the data file locally, and upload it to the folder "/content/MultiBench/data/"

In [ ]:
!mkdir data
%pip install gdown && gdown https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU

As Colab famously has bad handling of Conda env files, we'll install the dependencies manually so that it works. Please note that other systems might require installation of a long list of other dependencies.

## Tutorial

In [1]:
import torch
import sys
import os

# Specify the root project path directory
project_path = "D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/"
# Add directory to PYTHONPATH
sys.path.append(project_path)

### Load the datasets

First, we'll import and create the dataloader for the affective computing datasets (MUStARD, CMU-MOSI, UR-FUNNY, CMU-MOSEI), which we're working with:

You should check that you already downloaded the pkl files of each dataset into './MultiBench-main/examples/data/' directory

CMU-MOSEI: mosei_raw.pkl    https://drive.google.com/drive/folders/1A_hTmifi824gypelGobgl2M-5Rw9VWHv

CMU-MOSI: mosi_raw.pkl      https://drive.google.com/drive/folders/1uEK737LXB9jAlf9kyqRs6B9N6cDncodq

UR-FUNNY: humor.pkl         https://drive.google.com/drive/folders/1Agzm157lciMONHOHemHRSySmjn1ahHX1

MUStARD: sarcasm.pkl        https://drive.google.com/drive/folders/1JFcX-NF97zu9ZOZGALGU9kp8dwkP7aJ7

In [2]:
# Import the associated dataloader for affect datasets
from datasets.affect.get_data import get_dataloader

In [3]:
import os

def load_dataset(dataset_type):
    """
    :param dataset_type: string of the dataset you want to load

    The available dataset_type for each dataset correspond to:
    DATASET: dataset_type
    MOSEI: mosei_raw
    MOSI: mosi_raw
    FUNNY: humor
    MUSTARD: sarcasm
    """
    #data_path = '/content/MultiBench/data/' + dataset_type + '_raw.pkl' # if you are on Google Colab
    data_path = 'D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/data/' + dataset_type + '.pkl'
    
    if os.path.exists(data_path):
        print("OK, loading the dataset....", dataset_type)
    else:
        print("ERROR. file doesn't exist")
    print(data_path)

    try:
        traindata, validdata, testdata = get_dataloader(data_path, robust_test=False, max_pad=True, data_type=dataset_type, max_seq_len=50, batch_size=16)
        print("The dataset", dataset_type, "was successfully loaded.")
        return traindata, validdata, testdata
    except:    
        print("Please verify the dataset type you are trying to load.")

In [4]:
traindata_mosei, validdata_mosei, testdata_mosei = load_dataset("mosei_raw")

OK, loading the dataset.... mosei_raw
D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/data/mosei_raw.pkl
The dataset mosei_raw was successfully loaded.


In [5]:
traindata_mosi, validdata_mosi, testdata_mosi = load_dataset("mosi_raw")

OK, loading the dataset.... mosi_raw
D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/data/mosi_raw.pkl
The dataset mosi_raw was successfully loaded.


In [54]:
def checkInstances(traindata, validdata, testdata):
    #print("len:", len(traindata))
    batch_size_traindata = traindata.batch_size
    instances_traindata = len(traindata) * batch_size_traindata
    print("traindata instances:", instances_traindata)

    #print("len:", len(validdata))
    batch_size_validdata = validdata.batch_size
    instances_validdata = len(validdata) * batch_size_validdata
    print("validdata instances:", instances_validdata)

    #print("len:", len(testdata))
    batch_size_testdata = testdata.batch_size
    print("batch_size", batch_size_testdata)
    instances_testdata = len(testdata) * batch_size_testdata
    print("testdata instances:", instances_testdata,"\n")

print("MOSEI")
checkInstances(traindata_mosei, validdata_mosei, testdata_mosei) # type: ignore
print("MOSI")
checkInstances(traindata_mosi, validdata_mosi, testdata_mosi) # type: ignore
#print("FUNNY")
#checkInstances(traindata_funny, validdata_funny, testdata_funny) # type: ignore
#print("MUSTARD")
#checkInstances(traindata_mustard, validdata_mustard, testdata_mustard) # type: ignore

MOSEI
traindata instances: 16336
validdata instances: 1872
batch_size 16
testdata instances: 4672 

MOSI
traindata instances: 1296
validdata instances: 224
batch_size 16
testdata instances: 688 



### Model

Firstly, let's define the encoders of the raw modality information, which come from the "unimodals" section of MultiBench:

In [55]:
# Import several common modules .
from unimodals.common_models import Identity, Linear

#encoders = [Identity().cuda(), Identity().cuda(), Identity().cuda()]
encoders = [Linear(713, 128), Linear(74, 128), Linear(300, 128).cuda()]

Then, let's define the fusion paradigm, which will govern how we take the current modalities, and combine them.

For this example, we'll use the ConcatEarly fusion, which just concatenates the inputs along the second dimension.

In [56]:
# Import a fusion paradigm, in this case early concatenation.
from fusions.common_fusions import ConcatEarly  # noqa

# Initialize the fusion module
fusion = ConcatEarly().cuda()

In [57]:
import pickle

def checkModalitiesShape(dataset_type):
    print("Checking modalities shape for dataset:", dataset_type)
    #data_path = '/content/MultiBench/data/' + dataset_type + '_raw.pkl' # if you are on Google Colab
    data_path = 'D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/data/' + dataset_type + '.pkl'

    # loading the file_name.pkl
    with open(data_path, 'rb') as f:
        data = pickle.load(f)

    # Info about the file
    #print(type(data))  
    print('partitions:', data.keys())  
    print('partition keys:', data['train'].keys())  

    #print('id', data['train']['id'])
    #print('labels', data['train']['labels'])

    print('text', data['train']['text'][0].shape)
    print('audio', data['train']['audio'][0].shape)
    print('vision', data['train']['vision'][0].shape)

    input_dim = data['train']['text'][0].shape[1] + data['train']['audio'][0].shape[1] + data['train']['vision'][0].shape[1]
    print("input_dim (concat in 2nd dimension):", input_dim)

In [42]:
checkModalitiesShape("mosi_raw")

Checking modalities shape for dataset: mosi_raw
partitions: dict_keys(['train', 'valid', 'test'])
partition keys: dict_keys(['vision', 'audio', 'text', 'labels', 'id'])
text (50, 300)
audio (50, 74)
vision (50, 35)
input_dim (concat in 2nd dimension): 409


In [41]:
checkModalitiesShape("mosei_raw")

Checking modalities shape for dataset: mosei_raw
partitions: dict_keys(['train', 'valid', 'test'])
partition keys: dict_keys(['vision', 'audio', 'text', 'labels', 'id'])
text (50, 300)
audio (50, 74)
vision (50, 713)
input_dim (concat in 2nd dimension): 1087


Lastly, we'll define a 'head' module, which takes the output of the fusion module, and applies transformations to get an output that correponds to our problem

In [58]:
from unimodals.common_models import GRU, MLP, Sequential

input_dim = 128*3
head = Sequential(GRU(input_dim, 512, dropout=True, has_padding=False,batch_first=True, last_only=True), 
                  MLP(512, 512, 1)).cuda()

### Pretraining a model

In [ ]:
from importlib import reload
from training_structures import Supervised_Learning
from eval_scripts import performance
from datasets.affect import get_data

# Recargar el módulo Supervised_Learning
reload(Supervised_Learning)
reload(performance)
reload(get_data)

MultiBench's training loops, and set it running:

In [9]:
# Standard supervised learning training loop
from training_structures.Supervised_Learning import train

# For more information regarding parameters for any system, feel free to check out the documentation at multibench.readthedocs.io!
epochs = 5

dataset = 'mosei'
print("dataset:", dataset)
path_save_in = 'D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/pretrained/' + dataset + '_ef_r0_e' + str(epochs) + '_model.pt'
print(path_save_in)

traindata = traindata_mosei
validdata = validdata_mosei

train(encoders, fusion, head, traindata, validdata, epochs, task="regression", optimtype=torch.optim.AdamW,
      is_packed=False, lr=1e-3, save=path_save_in, weight_decay=0.01, objective=torch.nn.L1Loss())

dataset: mosei
D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/pretrained/mosei_ef_r0_e5_model.pt
MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output 

### Test the model

In [13]:
from training_structures.Supervised_Learning import MMDL
import os

print("For Testing:")

model_file = 'D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/pretrained/' + 'mosei_ef_r0_e5_model.pt'
# check if the file exists
if os.path.exists(model_file):
    print("OK, file exists:", model_file, "\n")
else:
    print("ERROR. file doesn't exist\n")

For Testing:
OK, file exists: D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/pretrained/mosei_ef_r0_e5_model.pt 



In [14]:
try:
    # Loading the model from a file previously saved
    model = torch.load(model_file)
    print("-----\nThe MODEL was successfully loaded. The configuration corresponds to:\n-----")
    print(model)
except:
    print("error")


-----
The MODEL was successfully loaded. The configuration corresponds to:
-----
MMDL(
  (encoders): ModuleList(
    (0): Linear(
      (fc): Linear(in_features=713, out_features=128, bias=True)
    )
    (1): Linear(
      (fc): Linear(in_features=74, out_features=128, bias=True)
    )
    (2): Linear(
      (fc): Linear(in_features=300, out_features=128, bias=True)
    )
  )
  (fuse): ConcatEarly()
  (head): Sequential(
    (0): GRU(
      (gru): GRU(384, 512, batch_first=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
    )
    (1): MLP(
      (fc): Linear(in_features=512, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=1, bias=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (lklu): LeakyReLU(negative_slope=0.2)
    )
  )
)


In [15]:
from training_structures.Supervised_Learning import test

print("Testing...")

testdata = testdata_mosei

test(model, testdata, 'affect', is_packed=False, criterion=torch.nn.L1Loss(), task="regression", no_robust=True) 

Testing...
MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 

MLP_output torch.Size([16, 1]) 


### Finetunning a model
The idea is to load an already pretrained model and finetuning it by training it with more data to adapt it to a certain task

In [16]:
import os

model_file = 'D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/pretrained/' + 'mosei_ef_r0_e5_model.pt'
# check if the file exists
if os.path.exists(model_file):
    print("OK, file exists:", model_file, "\n")
else:
    print("ERROR. file doesn't exist\n")



OK, file exists: D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/pretrained/mosei_ef_r0_e5_model.pt 



In [59]:
import os
import torch
import traceback

#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#os.environ['TORCH_USE_CUDA_DSA'] = '1'

try:
    # Loading the model from a file previously saved
    model = torch.load(model_file)
    print("-----\nThe MODEL was successfully loaded. The configuration corresponds to:\n-----")
    print(model)
    # Access to the model weights
    print("-----\nThe model WEIGHTS were successfully loaded:\n-----")
    weights = model.state_dict()
    print(weights)
    # Parameters
    print("-----\nThe model PARAMETERS were successfully loaded:\n-----")
    for name, tensor in weights.items():
        print(name, tensor.size())
except Exception as e:
    print("Error in model loading:", e)
    traceback.print_exc()

-----
The MODEL was successfully loaded. The configuration corresponds to:
-----
MMDL(
  (encoders): ModuleList(
    (0): Linear(
      (fc): Linear(in_features=713, out_features=128, bias=True)
    )
    (1): Linear(
      (fc): Linear(in_features=74, out_features=128, bias=True)
    )
    (2): Linear(
      (fc): Linear(in_features=300, out_features=128, bias=True)
    )
  )
  (fuse): ConcatEarly()
  (head): Sequential(
    (0): GRU(
      (gru): GRU(384, 512, batch_first=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
    )
    (1): MLP(
      (fc): Linear(in_features=512, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=1, bias=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (lklu): LeakyReLU(negative_slope=0.2)
    )
  )
)
-----
The model WEIGHTS were successfully loaded:
-----
OrderedDict([('encoders.0.fc.weight', tensor([[ 0.0023,  0.0102, -0.0081,  ..., -0.0049, -0.0406,  0.0025],
        [-0.0096,  0.0005,  0.0435,  ...

In [49]:
try:
    # Loading the model from a file previously saved
    model = torch.load(model_file)
    print("-----\nThe MODEL was successfully loaded. The configuration corresponds to:\n-----")
    print(model)
except:
    print("error")


-----
The MODEL was successfully loaded. The configuration corresponds to:
-----
MMDL(
  (encoders): ModuleList(
    (0): Linear(
      (fc): Linear(in_features=713, out_features=128, bias=True)
    )
    (1): Linear(
      (fc): Linear(in_features=74, out_features=128, bias=True)
    )
    (2): Linear(
      (fc): Linear(in_features=300, out_features=128, bias=True)
    )
  )
  (fuse): ConcatEarly()
  (head): Sequential(
    (0): GRU(
      (gru): GRU(384, 512, batch_first=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
    )
    (1): MLP(
      (fc): Linear(in_features=512, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=1, bias=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (lklu): LeakyReLU(negative_slope=0.2)
    )
  )
)


In [44]:
# Freezing the parameters
for param in model.parameters():
    param.requires_grad = False

You can modify certain layers of the "original" model

In [50]:
print("Names of Model Layers:")
for name, module in model.named_children():
    print(name)
    
model.encoders[0].fc.in_features = 35
#print(model.head[0].gru.input_size)

# update the input value
#new_input_value = 74
#model.head[0].gru.input_size = new_input_value

print("-----")
print(model)




Names of Model Layers:
encoders
fuse
head
-----
MMDL(
  (encoders): ModuleList(
    (0): Linear(
      (fc): Linear(in_features=35, out_features=128, bias=True)
    )
    (1): Linear(
      (fc): Linear(in_features=74, out_features=128, bias=True)
    )
    (2): Linear(
      (fc): Linear(in_features=300, out_features=128, bias=True)
    )
  )
  (fuse): ConcatEarly()
  (head): Sequential(
    (0): GRU(
      (gru): GRU(384, 512, batch_first=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
    )
    (1): MLP(
      (fc): Linear(in_features=512, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=1, bias=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (lklu): LeakyReLU(negative_slope=0.2)
    )
  )
)


Then you use the model to train it now with other dataset for a different task

In [ ]:
import torch
from training_structures.Supervised_Learning import train

dataset = 'mosi'

traindata = traindata_mosi
validdata = validdata_mosi
epochs = 5

#optimizer = torch.optim.AdamW(model.parameters()) # train torch.optim.AdamW
#criterion = torch.nn.MSELoss()

path_save_in = 'D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/finetuned/' + dataset + '_ef_r0_e' + str(epochs) + '_model.pt'

#train(encoders, fusion, head, traindata, validdata, epochs, task="regression", optimtype=torch.optim.AdamW,
#      is_packed=False, lr=0.001, save=path_save_in, weight_decay=0.01, objective=torch.nn.L1Loss())
train(encoders, fusion, head, traindata, validdata, epochs, task="regression", optimtype=torch.optim.AdamW,
      is_packed=False, lr=1e-3, save=path_save_in, weight_decay=0.01, objective=torch.nn.L1Loss())

### Testing the fine tuned model

In [ ]:
from training_structures.Supervised_Learning import MMDL
import os

print("For Testing:")

model_file = 'D:/itzel/PhD Computer Science/03 codes/codes-references/MultiBench-main/examples/finetuned/' + 'mosi_ef_r0_e5_model.pt'
# check if the file exists
if os.path.exists(model_file):
    print("OK, file exists:", model_file, "\n")
else:
    print("ERROR. file doesn't exist\n")

In [ ]:
from training_structures.Supervised_Learning import test

print("Testing...")

testdata = testdata_mosi

test(model, testdata, 'affect', is_packed=False, criterion=torch.nn.L1Loss(), task="regression", no_robust=True) 